# Arquitetura de Agente Autônomo v3: Refatoração Granular

Este notebook implementa a versão 3 do agente autônomo, focando em uma refatoração granular para máxima clareza e modularidade. Cada função e cada método da classe `Agent` serão separados em suas próprias células, com explicações detalhadas e independentes.

O objetivo é criar um documento que não apenas execute a lógica do agente, mas que também sirva como uma documentação técnica profunda de cada componente individual, facilitando a manutenção, o aprendizado e a expansão futura do sistema.

O ciclo **OODA (Observe, Orient, Decide, Act)** continua sendo o pilar da arquitetura, agora com cada passo explicitamente isolado em sua própria célula de código.

### 1. Template da Ficha de Personagem

A função `create_character_sheet` é a fundação do estado do agente. Ela gera um dicionário Python que serve como um template em branco para a ficha de um personagem.

**Funcionamento:**
- **Estrutura Hierárquica**: A ficha é organizada em seções como `info`, `resources`, `skills`, e `status`, facilitando o acesso e a modificação de atributos específicos.
- **Valores Padrão**: Todos os campos são inicializados com valores padrão (zeros, listas vazias ou `None`). Isso garante que o agente sempre tenha uma estrutura de dados consistente, mesmo antes da personalização.
- **Pronta para Expansão**: A estrutura inclui seções como `modifiers` e `expert` skills, que são projetadas para serem preenchidas dinamicamente durante o jogo, à medida que o personagem ganha novas habilidades ou sofre penalidades.

In [ ]:
import random

def create_character_sheet():
    """Cria um template para a ficha de personagem."""
    return {
        "info": {
            "name": "Character Name",
            "occupation": None,
            "age": 30,
            "backstory": ""
        },
        "contacts": {},
        "case_files": [],
        "magic": {"spells": [], "signare": []},
        "characteristics": {
            "STR": {"full": 0, "half": 0}, "CON": {"full": 0, "half": 0},
            "DEX": {"full": 0, "half": 0}, "INT": {"full": 0, "half": 0},
            "POW": {"full": 0, "half": 0}
        },
        "resources": {
            "luck": {"starting": 0, "current": 0},
            "magic_pts": {"starting": 0, "current": 0},
            "mov": 8
        },
        "skills": {
            "common": {
                "Athletics": {"full": 30, "half": 15}, "Drive": {"full": 30, "half": 15},
                "Navigate": {"full": 30, "half": 15}, "Observation": {"full": 30, "half": 15},
                "Read Person": {"full": 30, "half": 15}, "Research": {"full": 30, "half": 15},
                "Social": {"full": 30, "half": 15}, "Stealth": {"full": 30, "half": 15},
            },
            "combat": {
                "Fighting": {"full": 30, "half": 15}, "Firearms": {"full": 30, "half": 15}
            },
            "expert": {}
        },
        "status": {
            "damage_levels": ["Hurt", "Bloodied", "Down", "Impaired"],
            "current_damage": None,
            "damage_taken": 0,
            "modifiers": []  # e.g., {"skill": "Fighting", "type": "penalty_die", "duration": "scene"}
        },
        "inventory": {"equipment": [], "weapons": []},
        "page_history": []
    }

### 2. Configuração Inicial do Personagem

A função `setup_character` personaliza a ficha de personagem criada anteriormente. Ela recebe a ficha em branco e os dados básicos do personagem (nome, ocupação) e a preenche com os valores iniciais corretos.

**Funcionamento:**
- **Dados Básicos**: Preenche as informações de identidade do personagem.
- **Recursos Iniciais**: Calcula e define recursos iniciais, como a Sorte (`luck`), usando uma rolagem de dados para adicionar variabilidade.
- **Perícias de Ocupação**: A lógica principal está em ajustar as perícias (`skills`) com base na `occupation` escolhida. Isso simula a especialização de uma classe de personagem em um RPG, dando bônus em perícias relevantes (ex: um `Police Officer` tem `Law` e `Fighting` aprimorados).

In [ ]:
def setup_character(sheet, name, occupation, backstory):
    """Configura a ficha de personagem com base na ocupação e história."""
    sheet["info"]["name"] = name
    sheet["info"]["occupation"] = occupation
    sheet["info"]["backstory"] = backstory

    # Define a sorte inicial do personagem
    luck_roll = random.randint(1, 10) + random.randint(1, 10) + 50
    sheet["resources"]["luck"]["starting"] = luck_roll
    sheet["resources"]["luck"]["current"] = luck_roll

    # Ajusta as perícias com base na ocupação
    if occupation == "Police Officer":
        for skill in ["Law", "Social", "Athletics", "Fighting"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Law"] = {"full": 60, "half": 30}
    elif occupation == "Social Worker":
        for skill in ["Observation", "Research", "Social"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}
    elif occupation == "Nurse":
        for skill in ["Observation", "Read Person", "Social"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Medicine"] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}

    return sheet

### 3. Mecânica de Rolagem de Dados (D100)

A função `make_check` é o motor de resolução de ações do RPG. Ela simula uma rolagem de dado de 100 lados (D100) e determina o nível de sucesso de uma ação com base no valor de uma perícia.

**Funcionamento:**
- **Rolagem D100**: Simula uma rolagem de percentil combinando um dado de dezenas e um de unidades.
- **Bônus e Penalidade**: Implementa a mecânica de "vantagem" e "desvantagem". Com um dado de bônus (`bonus_dice`), o menor de dois dados de dezena é usado. Com um dado de penalidade (`penalty_die`), o maior é usado. Se ambos estiverem presentes, eles se anulam.
- **Níveis de Sucesso**: O resultado da rolagem é comparado com o valor da perícia (`target_value`) e sua metade (`half_value`) para determinar um de cinco níveis de sucesso, retornados como um valor numérico:
    - **5**: Sucesso Crítico (rolagem 1)
    - **4**: Sucesso Difícil (rolagem <= metade da perícia)
    - **3**: Sucesso Normal (rolagem <= perícia)
    - **2**: Falha (rolagem > perícia)
    - **1**: Falha Crítica (rolagem 100)
- **Saída Declarativa**: O retorno numérico (`level`) permite que o sistema encontre a consequência correta de forma declarativa, sem precisar de lógica complexa no código que chama a função.

In [ ]:
def make_check(target_value, half_value, bonus_dice=False, penalty_die=False):
    """Realiza um teste de perícia D100 e retorna o nível de sucesso numérico."""
    tens_roll_1 = random.randint(0, 9) * 10
    tens_roll_2 = random.randint(0, 9) * 10
    units_roll = random.randint(1, 10)

    # Um dado de bônus e um dado de penalidade se anulam.
    if bonus_dice and penalty_die:
        bonus_dice = False
        penalty_die = False

    if bonus_dice:
        final_tens = min(tens_roll_1, tens_roll_2)
        print("Applied bonus die.")
    elif penalty_die:
        final_tens = max(tens_roll_1, tens_roll_2)
        print("Applied penalty die.")
    else:
        final_tens = tens_roll_1

    # Calcula o resultado final da rolagem
    if final_tens == 0 and units_roll == 10:
        final_roll = 100
    elif final_tens == 0:
        final_roll = units_roll
    else:
        final_roll = final_tens + (units_roll % 10)

    # Determina o nível de sucesso
    if final_roll == 1:
        return (5, final_roll)  # Critical Success
    if final_roll == 100:
        return (1, final_roll)  # Fumble
    if final_roll <= half_value:
        return (4, final_roll)  # Hard Success
    if final_roll <= target_value:
        return (3, final_roll)  # Success
    
    return (2, final_roll)  # Failure

### 4. A Classe `Agent`: O Coração do Sistema

A classe `Agent` encapsula toda a lógica de estado, decisão e ação, funcionando como a "espinha dorsal" do jogador autônomo. A seguir, seus métodos serão detalhados um a um.

#### 4.1. Construtor (`__init__`) e Representação (`__repr__`)

**`__init__(self, name, occupation, game_instructions, game_data)`**
- **Propósito**: Inicializar o agente e seu estado interno.
- **Funcionamento**:
    1.  Chama `create_character_sheet()` para obter um template de ficha de personagem.
    2.  Chama `setup_character()` para personalizar a ficha com os dados do personagem (nome, ocupação, etc.).
    3.  Armazena as dependências injetadas (`game_data`, `game_instructions`) como atributos da instância.
    4.  Define o estado inicial do jogo, como a `current_page` (página inicial) e o `combat_status`.

**`__repr__(self)`**
- **Propósito**: Fornecer uma representação textual clara e concisa do objeto `Agent`.
- **Funcionamento**: Retorna uma string que mostra o nome e a ocupação do agente, útil para logs e debugging.

In [ ]:
class Agent:
    """
    Implementa um agente autônomo que navega por um livro-jogo
    seguindo o ciclo OODA (Observe, Orient, Decide, Act).
    """
    
    def __init__(self, name, occupation, game_instructions, game_data):
        """
        Inicializa o agente, criando sua ficha de personagem e configurando o estado inicial.
        """
        base_sheet = create_character_sheet()
        self.sheet = setup_character(base_sheet, name, occupation, game_instructions.get_backstory())
        self.game_data = game_data
        self.current_page = 1
        self.combat_status = {}

    def __repr__(self):
        """Retorna uma representação textual do agente."""
        return f"Agent(Name: {self.sheet['info']['name']}, Occupation: {self.sheet['info']['occupation']})"

#### 4.2. O Método de Decisão (`_llm_decide`)

**`_llm_decide(self, choices)`**
- **Propósito**: Simular o processo de "Decisão" do ciclo OODA. Este método avalia as opções disponíveis (`choices`) e seleciona a mais apropriada com base no estado atual do agente e em sua lógica interna.
- **Funcionamento**:
    1.  **Itera sobre as Opções**: Percorre cada `choice` fornecida na página atual do livro-jogo.
    2.  **Verifica Pré-requisitos (`requires`)**: Para cada opção, verifica se existe um campo `requires`. Este campo define as condições necessárias para que a opção seja válida.
        - **Condições de Atributo**: Pode checar o valor de um atributo na ficha do personagem (ex: `"occupation": "Police Officer"`).
        - **Condições de Estado**: Pode checar o estado do jogo (ex: dano sofrido, `"damage_taken": {"min": 1}`).
    3.  **Lógica de Seleção**:
        - Se uma opção não tem `requires`, ela é considerada uma "opção padrão" ou "catch-all" e é armazenada como fallback.
        - Se uma opção tem `requires` e todas as condições são atendidas pelo estado atual do agente, essa opção é escolhida imediatamente e o método retorna.
    4.  **Retorno**: Se nenhuma opção com pré-requisitos for satisfeita, o método retorna a opção padrão que foi armazenada. Isso garante que o agente sempre tome uma decisão.
- **Design Declarativo**: A refatoração para usar o campo `requires` tornou a lógica de decisão puramente declarativa. O método não precisa mais interpretar strings de texto; ele apenas compara dados estruturados, tornando o sistema mais robusto e fácil de manter.

In [ ]:
    def _llm_decide(self, choices):
        """
        Decide qual ação tomar com base nas opções e no estado do agente.
        Esta versão usa uma lógica puramente declarativa baseada no campo 'requires'.
        """
        default_choice = None
        
        for choice in choices:
            # Se a escolha não tiver pré-requisitos, é a padrão
            if "requires" not in choice:
                if not default_choice:  # Armazena a primeira padrão encontrada
                    default_choice = choice
                continue

            # Avalia as condições em 'requires'
            conditions_met = True
            for key, value in choice["requires"].items():
                # Condição de ocupação
                if key == "occupation" and self.sheet["info"]["occupation"] != value:
                    conditions_met = False
                    break
                # Condição de dano
                elif key == "damage_taken":
                    min_damage = value.get("min", 0)
                    max_damage = value.get("max", float('inf'))
                    if not (min_damage <= self.sheet["status"]["damage_taken"] <= max_damage):
                        conditions_met = False
                        break
                # Adicionar outras verificações de condição aqui, se necessário
            
            # Se todas as condições forem atendidas, escolhe esta opção
            if conditions_met:
                print(f"Agente decidiu com base em pré-requisitos: {choice['requires']}")
                return choice

        # Se nenhuma escolha com pré-requisitos foi satisfeita, usa a padrão
        if default_choice:
            print("Agente decidiu pela opção padrão.")
            return default_choice
        
        # Fallback de segurança (não deve acontecer em um livro-jogo bem formado)
        print("Nenhuma opção válida encontrada, retornando a primeira da lista.")
        return choices[0]

#### 4.3. O Método de Ação (`perform_action`)

**`perform_action(self, choice)`**
- **Propósito**: Executar a "Ação" do ciclo OODA. Este método pega a `choice` selecionada pelo `_llm_decide` e aplica seus efeitos no estado do agente e do jogo.
- **Funcionamento**:
    1.  **Aplica Efeitos**: Verifica se a `choice` tem um campo `effects`. Se tiver, aplica os custos ou mudanças ao estado do agente. Atualmente, ele suporta o gasto de pontos de `luck`.
    2.  **Executa Rolagens de Dados**:
        - Se a `choice` contém um campo `roll`, significa que uma rolagem de perícia é necessária. O método identifica a perícia, busca o valor correspondente na ficha do personagem e chama `make_check()` para obter o nível de sucesso.
        - O resultado da rolagem (o `level`) é usado como chave para encontrar a consequência correta no dicionário `results` da `choice`.
    3.  **Processa Resultados**:
        - O `outcome` (a consequência textual da ação) é impresso na tela.
        - Se o resultado tiver um campo `goto`, o método atualiza a `current_page` do agente, fazendo o jogo avançar.
    4.  **Navegação Direta**: Se a `choice` não exige uma rolagem, ela provavelmente tem um `goto` direto, que é usado para navegar para a próxima página.
- **Retorno**: O método retorna o `outcome` (a descrição do que aconteceu) para que o loop principal do jogo possa exibi-lo.

In [ ]:
    def perform_action(self, choice):
        """
        Executa a ação decidida, aplicando efeitos e rolagens de dados.
        """
        outcome = ""
        
        # 1. Aplicar efeitos (custos) da ação
        if "effects" in choice:
            if "luck" in choice["effects"]:
                cost = choice["effects"]["luck"]
                self.sheet["resources"]["luck"]["current"] -= cost
                print(f"Spent {cost} luck. Current luck: {self.sheet['resources']['luck']['current']}")

        # 2. Executar rolagens de dados, se necessário
        if "roll" in choice:
            skill_to_roll = choice["roll"]
            
            # Encontra a perícia na ficha do personagem (comum, combate ou expert)
            if skill_to_roll in self.sheet["skills"]["common"]:
                skill_values = self.sheet["skills"]["common"][skill_to_roll]
            elif skill_to_roll in self.sheet["skills"]["combat"]:
                skill_values = self.sheet["skills"]["combat"][skill_to_roll]
            else:
                skill_values = self.sheet["skills"]["expert"].get(skill_to_roll, {"full": 0, "half": 0})

            # Realiza o teste
            level, roll_value = make_check(skill_values["full"], skill_values["half"])
            print(f"Rolled {skill_to_roll}: {roll_value} vs {skill_values['full']} -> Level {level}")

            # Processa o resultado da rolagem
            result = choice["results"].get(str(level))
            if result:
                outcome = result["outcome"]
                if "goto" in result:
                    self.current_page = result["goto"]
        
        # 3. Navegação direta (sem rolagem)
        elif "goto" in choice:
            self.current_page = choice["goto"]
            outcome = choice.get("outcome", "")

        return outcome

#### 4.4. O Loop Principal do Agente (`run`)

**`run(self)`**
- **Propósito**: Orquestrar o ciclo OODA completo, servindo como o ponto de entrada para a execução do agente no livro-jogo.
- **Funcionamento**:
    1.  **Loop Infinito (Controlado)**: O método opera dentro de um `while True`, que continua executando o ciclo do agente até que uma condição de término seja alcançada (uma página sem opções ou um `goto` para `0`).
    2.  **Observação (`_observe`)**: No início de cada ciclo, chama o método `_observe` (a ser detalhado a seguir) para obter o estado atual do jogo (o texto e as opções da página atual).
    3.  **Orientação (`_orient`)**: Em seguida, chama `_orient` para processar as informações observadas e prepará-las para a decisão.
    4.  **Decisão (`_llm_decide`)**: Passa as `choices` para o `_llm_decide`, que seleciona a melhor ação.
    5.  **Ação (`perform_action`)**: Executa a `choice` selecionada através do `perform_action` e armazena o `outcome`.
    6.  **Condição de Parada**: O loop é interrompido se a `current_page` for `0` ou se a página atual não tiver `choices`, indicando o fim de um arco da história.
- **Ciclo Contínuo**: Este método encapsula a autonomia do agente, permitindo que ele navegue pela história de forma independente, página por página, até encontrar um ponto final.

In [ ]:
    def run(self):
        """
        Executa o ciclo OODA principal para navegar pelo livro-jogo.
        """
        while True:
            # 1. Observe
            page_text, choices = self._observe()
            if not choices:
                print("Fim da história (nenhuma escolha encontrada).")
                break

            # 2. Orient
            self._orient(page_text)
            
            # 3. Decide
            chosen_action = self._llm_decide(choices)
            
            # 4. Act
            outcome = self.perform_action(chosen_action)
            print(f"Resultado: {outcome}\n---")

            # Condição de parada
            if self.current_page == 0:
                print("Fim da história (goto: 0).")
                break

#### 4.5. Métodos Auxiliares: `_observe` e `_orient`

Estes dois métodos completam as fases restantes do ciclo OODA, focando na coleta e processamento de informações.

**`_observe(self)`**
- **Propósito**: Implementar a fase de "Observação". O método coleta todos os dados brutos da página atual do livro-jogo.
- **Funcionamento**:
    1.  Acessa o `self.game_data` usando a `self.current_page` como chave.
    2.  Extrai o texto da página (`page_text`) e a lista de `choices` disponíveis.
    3.  Imprime o número da página e seu texto para que o usuário (ou desenvolvedor) possa acompanhar a jornada do agente.
- **Retorno**: Retorna uma tupla contendo `(page_text, choices)`.

**`_orient(self, page_text)`**
- **Propósito**: Implementar a fase de "Orientação". O método processa as informações brutas observadas e atualiza o estado interno do agente.
- **Funcionamento**:
    1.  Adiciona a `current_page` ao histórico de páginas visitadas (`page_history`) na ficha do personagem. Isso cria um registro do caminho percorrido pelo agente.
    2.  O `page_text` é passado como argumento, permitindo futuras expansões onde o agente poderia, por exemplo, usar um LLM para extrair informações contextuais do texto da página e atualizar sua "compreensão" da situação. Atualmente, esta função é simples, mas está projetada para ser extensível.

In [ ]:
    def _observe(self):
        """
        Observa o ambiente, lendo o texto e as opções da página atual.
        """
        page_data = self.game_data.get(str(self.current_page), {})
        page_text = page_data.get("text", "Página não encontrada.")
        choices = page_data.get("choices", [])
        
        print(f"--- Página {self.current_page} ---\n{page_text}\n")
        return page_text, choices

    def _orient(self, page_text):
        """
        Orienta o agente, atualizando seu estado interno com base nas observações.
        """
        # Adiciona a página atual ao histórico
        if self.current_page not in self.sheet["page_history"]:
            self.sheet["page_history"].append((self.current_page,self.page_text))
        
        # Futuramente, poderia usar um LLM para extrair contexto do page_text
        pass

### 5. Execução Principal: Configurando e Rodando o Agente

Esta é a célula final que une todas as peças. Ela é responsável por:

1.  **Definir as Instruções do Jogo**: Cria um dicionário `game_instructions` que contém a história de fundo (`backstory`) do personagem. Em um sistema mais complexo, isso poderia incluir regras gerais, objetivos de longo prazo, etc.
2.  **Definir os Dados do Jogo**: O dicionário `game_data` representa o "livro-jogo" em si. Cada chave é o número de uma página, e o valor contém o texto da página e as `choices` (opções) disponíveis. É aqui que a estrutura declarativa brilha, com cada `choice` contendo toda a lógica necessária para a sua resolução (`requires`, `roll`, `goto`, `results`).
3.  **Instanciar o Agente**: Cria uma instância da classe `Agent`, injetando as `game_instructions` e o `game_data`. O nome e a ocupação do agente são definidos aqui.
4.  **Executar o Agente**: Chama o método `agent.run()`, que inicia o ciclo OODA e faz o agente navegar autonomamente pela história definida no `game_data`.

In [ ]:
# Definição das instruções e dados do jogo
game_instructions = {
    "get_backstory": lambda: "Um oficial recém-formado na divisão de crimes especiais, conhecido por sua abordagem metódica."
}
import pages
game_data = 

# Instancia e executa o agente
agent = Agent(name="Alex", occupation="Police Officer", game_instructions=game_instructions, game_data=game_data)
agent.run()